In this notebook, we create another version of the network visualization, this time using the Networkx package.

We directly load the clean_connections.csv file that we built during data cleaning as a graph.  Then, we add HoverTools and TapTools to allow us to mouse over the nodes and see the highlighted edges corresponding to that node.

The output is saved as network3.html.

## Importing packages and data

In [1]:
import networkx as nx

from bokeh.models import (Circle, HoverTool, MultiLine,
                          NodesAndLinkedEdges,EdgesAndLinkedNodes, Plot, Range1d, TapTool,
                         LinearAxis)
from bokeh.plotting import from_networkx, show

from bokeh.models import CustomJS

In [2]:
import matplotlib.pyplot as plt

In [3]:
from bokeh.plotting import output_notebook, figure

In [4]:
from bokeh.io import save, output_file

In [5]:
import pandas as pd

In [7]:
connections=pd.read_csv('clean_connections.csv')

In [8]:
people=pd.read_csv('registeredparticipants.csv', encoding='latin1')

## Making the graph

In [9]:
my_graph=nx.from_pandas_edgelist(connections, source='name', target='conn')

In [13]:
labels={}
for i in my_graph.nodes:
    count=list(connections['name'].values).count(i) + list(connections['conn'].values).count(i)
    labels[i]= {'Name':people.iloc[i]['first'].strip() + ' ' + people.iloc[i]['last'].strip(),
                'Connections' :  count}

In [11]:
output_file('network3.html')

In [12]:
G = my_graph

nx.set_node_attributes(G, labels)

plot = Plot(width=500, height=500,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

plot.title.text = "USTARS 2023 Connections"

tooltips=[('Name', '@Name'),
         ('Connections','@Connections')]

callback = CustomJS(args={'lab': 'lab'},
                    code="""
    var tooltips = document.getElementsByClassName("bk-tooltip");
    for (var i = 0; i < tooltips.length; i++) {
        tooltips[i].style.top = '25px'; 
        tooltips[i].style.left = 0 + 'px'; 
        tooltips[i].style.width = 200 + 'px';
    } """)

plot.add_tools(HoverTool(tooltips=tooltips, 
                         callback=callback, 
                         show_arrow=False,
                        attachment='below'),
               TapTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='blue')
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color='gold')
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color='yellow')

graph_renderer.edge_renderer.glyph = MultiLine(line_color="dimgray", line_alpha=0.8, line_width=2)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_alpha=1, line_color='gold', line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color='yellow', line_width=5)


graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.selection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

show(plot)